# Generating Data

In [1]:
import numpy as np
import tensorflow as tf
import pgmpy
from GPflow.gpr import GPR
from GPflow.kernels import RBF
from tensorflow.contrib.crf import viterbi_decode

from matplotlib import pyplot as plt

%matplotlib inline

## Sampling potentials

In [2]:
n_seq = 150
D = 2
n_labels = 2
max_len = 5

x = np.random.normal(size=(n_seq, max_len, D)) / 3

In [3]:
f_bin = np.random.multivariate_normal(mean=np.array([1, 0., 0., 1]), cov=np.eye(4) * 0.1, size=n_seq)

In [4]:
x_flat = x.reshape([-1, D])
x_init = np.zeros([1, D])
f_init = np.zeros([1, 1])

In [5]:
gp_0 = GPR(x_init, f_init, kern=RBF(input_dim=D, lengthscales=[0.3]))
gp_1 = GPR(x_init, f_init, kern=RBF(input_dim=D, lengthscales=[0.2]))

In [6]:
mean_0, var_0 = gp_0.predict_f_full_cov(x_flat)
var_0 += np.eye(mean_0.size)[:, :, None] * 0.05**2

mean_1, var_1 = gp_1.predict_f_full_cov(x_flat)
var_1 += np.eye(mean_1.size)[:, :, None] * 0.05**2

f_un_flat_0 = np.random.multivariate_normal(mean_0[:, 0], var_0[:, :, 0])
f_un_flat_1 = np.random.multivariate_normal(mean_1[:, 0], var_1[:, :, 0])

In [7]:
# plt.plot(x_flat[:, 0], f_un_flat_0, 'bo')
# plt.plot(x_flat[:, 0], f_un_flat_1, 'ro')

In [8]:
f_un_0 = f_un_flat_0.reshape(x.shape[:-1])
f_un_1 = f_un_flat_1.reshape(x.shape[:-1])

f_un = np.dstack([f_un_0[:, :, None], f_un_1[:, :, None]])

## Sampling labels

In [9]:
f_bin = f_bin.reshape([n_seq, 2, 2])

In [19]:
y = []
for seq in range(n_seq):
    y_ = viterbi_decode(f_un[seq], f_bin[seq])[0]
    y.append(y_)
y = np.array(y)

In [21]:
f_un[seq].shape

(5, 2)

In [13]:
x_tr, y_tr = x[:100], y[:100]
x_te, y_te = x[100:], y[100:]

In [18]:
# np.save('data/x_tr', x_tr)
# np.save('data/y_tr', y_tr)
# np.save('data/x_te', x_te)
# np.save('data/y_te', y_te)